In [1]:
import time

import torch

from modules.Dcls2d import Dcls2d

assert torch.cuda.is_available()
cuda_device = torch.device("cuda")  # device object representing GPU

in_channels = 32
out_channels = 64
kernel_size = (3,3)
dilation = (1,1)
stride = (1,1)
padding = (1,1)
groups = 1
bias = False

m = torch.nn.Conv2d(in_channels=in_channels,
              out_channels=out_channels,
              kernel_size=kernel_size,
              dilation=dilation,
              stride=stride,
              padding=padding,
              groups=groups,
              bias=bias).to(cuda_device)

n = Dcls2d(in_channels=in_channels,
              out_channels=out_channels,
              kernel_size=kernel_size,
              dilation=dilation,
              stride=stride,
              padding=padding,
              groups=groups,
              bias=bias).to(cuda_device)

X = torch.randn(100, in_channels,54,87,device=cuda_device) 


In [2]:
print(X.size())

print(m(X).size())
print(n(X).size())



torch.Size([100, 32, 54, 87])
torch.Size([100, 64, 54, 87])
torch.Size([100, 64, 54, 87])


In [3]:
forward = 0
backward = 0
for _ in range(100):
    start = time.time()
    a = m(X)
    torch.cuda.synchronize()
    forward += time.time() - start

    start = time.time()
    (a.sum()).backward()
    torch.cuda.synchronize()
    backward += time.time() - start

print('Forward: {:.3f} us | Backward {:.3f} us'.format(forward * 1e6/1e2, backward * 1e6/1e2))

Forward: 2827.787 us | Backward 6429.150 us


In [4]:
forward = 0
backward = 0
for _ in range(100):
    start = time.time()
    b = n(X)
    torch.cuda.synchronize()
    forward += time.time() - start

    start = time.time()
    (b.sum() ).backward()
    torch.cuda.synchronize()
    backward += time.time() - start

print('Forward: {:.3f} us | Backward {:.3f} us'.format(forward * 1e6/1e2, backward * 1e6/1e2))

Forward: 7908.678 us | Backward 2496.898 us


In [5]:
forward = 0
backward = 0
for _ in range(100):
    start = time.time()
    c = n(X)
    torch.cuda.synchronize()
    forward += time.time() - start

    start = time.time()
    (c.sum() ).backward()
    torch.cuda.synchronize()
    backward += time.time() - start

print('Forward: {:.3f} us | Backward {:.3f} us'.format(forward * 1e6/1e2, backward * 1e6/1e2))

Forward: 7187.433 us | Backward 2093.401 us
